In [1]:
pip install xgboost optuna

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 3.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 13.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns

from scipy.stats import rankdata
from sklearn.cluster import KMeans
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, roc_auc_score, cohen_kappa_score, davies_bouldin_score, calinski_harabasz_score, silhouette_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVR
# from lightgbm import LGBMClassifier, LGBMRegressor
from xgboost import XGBClassifier, XGBRegressor
# from catboost import CatBoostClassifier

import optuna 

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/TS-S3-Ep5/train.csv'
file_key_2 = 'Tabular-Playground-Series/TS-S3-Ep5/test.csv'
file_key_3 = 'Tabular-Playground-Series/TS-S3-Ep5/sample_submission.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

## Reading data files
train = pd.read_csv(file_content_stream_1)
test = pd.read_csv(file_content_stream_2)
submission = pd.read_csv(file_content_stream_3)

## Enginering features
train['alcohol_density'] = train['alcohol'] * train['density']
train['sulphate/density'] = train['sulphates']  / train['density']
train['alcohol_sulphate'] = train['alcohol'] * train['sulphates']

test['alcohol_density'] = test['alcohol']  * test['density']
test['sulphate/density'] = test['sulphates']  / test['density']
test['alcohol_sulphate'] = test['alcohol'] * test['sulphates']

test_md = test.copy()

X = train[['sulphate/density', 'alcohol_density', 'alcohol', 'sulphates', 'fixed acidity', 'alcohol_sulphate']]
Y = train['quality'] 

test_md = test_md[['sulphate/density', 'alcohol_density', 'alcohol', 'sulphates', 'fixed acidity', 'alcohol_sulphate']]

In [2]:
train.head()

,Id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,alcohol_density,sulphate/density,alcohol_sulphate
0,0,8.0,0.50,0.39,2.2,0.073,30.0,39.0,0.99572,3.33,0.77,12.1,6,12.048212,0.773310,9.317
1,1,9.3,0.30,0.73,2.3,0.092,30.0,67.0,0.99854,3.32,0.67,12.8,6,12.781312,0.670980,8.576
2,2,7.1,0.51,0.03,2.1,0.059,3.0,12.0,0.99660,3.52,0.73,11.3,7,11.261580,0.732490,8.249
3,3,8.1,0.87,0.22,2.6,0.084,11.0,65.0,0.99730,3.20,0.53,9.8,5,9.773540,0.531435,5.194
4,4,8.5,0.36,0.30,2.3,0.079,10.0,45.0,0.99444,3.20,1.36,9.5,6,9.447180,1.367604,12.920


# Optimal Rounder

In [3]:
from functools import partial
import numpy as np
import scipy as sp

class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 3
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 4
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 5
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 6
            elif pred >= coef[3] and pred < coef[4]:
                X_p[i] = 7
            else:
                X_p[i] = 8

        ll = cohen_kappa_score(y, X_p, weights = 'quadratic')
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X = X, y = y)
        initial_coef = [3.5, 4.5, 5.5, 6.5, 7.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method = 'nelder-mead')

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 3
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 4
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 5
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 6
            elif pred >= coef[3] and pred < coef[4]:
                X_p[i] = 7
            else:
                X_p[i] = 8
        return X_p

    def coefficients(self):
        return self.coef_['x']

# Optuna Optimization

In [4]:
def objective(trial):
    
    ## Parameters to be evaluated
    param = dict(objective = 'reg:absoluteerror',
                 eval_metric = 'mae',
                 tree_method = 'hist', 
                 random_state = 42,
                 max_depth = trial.suggest_int('max_depth', 2, 10),
                 learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log = True),
                 n_estimators = trial.suggest_int('n_estimators', 30, 10000),
                 gamma = trial.suggest_float('gamma', 0, 10),
                 min_child_weight = trial.suggest_int('min_child_weight', 1, 100),
                 colsample_bytree = trial.suggest_float('colsample_bytree', 0.2, 0.9),
                 subsample = trial.suggest_float('subsample', 0.2, 0.9)
                )

    scores = []

#     skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
    skf = KFold(n_splits = 5, shuffle = True, random_state = 42)
    
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, Y)):
        
        print(fold, end = ' ')
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train , y_valid = Y.iloc[train_idx] , Y.iloc[valid_idx]

        model = XGBRegressor(**param)
        model.fit(X_train, y_train)
        
        preds_train = model.predict(X_train)
        preds_valid = model.predict(X_valid)
        
        optR = OptimizedRounder()
        optR.fit(preds_train, y_train)
        coef = optR.coefficients()
        preds_valid = optR.predict(preds_valid, coef).astype(int)
        
        score = cohen_kappa_score(y_valid,  preds_valid, weights = "quadratic")
        scores.append(score)
        
    return np.mean(scores)

In [ ]:
study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials = 50, timeout = 3600)

[I 2023-02-10 22:34:22,748] A new study created in memory with name: no-name-87837fb2-6233-4e40-8af9-d690e2b42c12


0 1 2 3 4 

[I 2023-02-10 22:35:00,408] Trial 0 finished with value: 0.37683807893640325 and parameters: {'max_depth': 7, 'learning_rate': 0.001927203039860696, 'n_estimators': 6488, 'gamma': 3.0726263617277216, 'min_child_weight': 6, 'colsample_bytree': 0.7092265933084838, 'subsample': 0.6438561603099846}. Best is trial 0 with value: 0.37683807893640325.


0 1 2 3 4 

[I 2023-02-10 22:35:16,812] Trial 1 finished with value: 0.30859899214122266 and parameters: {'max_depth': 3, 'learning_rate': 0.0004286353510303723, 'n_estimators': 63, 'gamma': 1.1379063234718223, 'min_child_weight': 2, 'colsample_bytree': 0.699821750617027, 'subsample': 0.24577246280135367}. Best is trial 0 with value: 0.37683807893640325.


...

0 1 2 3 4 

[I 2023-02-10 22:50:25,079] Trial 30 finished with value: 0.471611008768662 and parameters: {'max_depth': 7, 'learning_rate': 0.005253228451442322, 'n_estimators': 8634, 'gamma': 9.389571177093092, 'min_child_weight': 70, 'colsample_bytree': 0.25425277311963057, 'subsample': 0.6126069164825733}. Best is trial 28 with value: 0.526084189524658.


0 1 2 3 4 

[I 2023-02-10 22:50:53,012] Trial 31 finished with value: 0.4902423015651432 and parameters: {'max_depth': 7, 'learning_rate': 0.0023099098646266747, 'n_estimators': 6998, 'gamma': 7.774598217229545, 'min_child_weight': 55, 'colsample_bytree': 0.31758789004414323, 'subsample': 0.5292101562706857}. Best is trial 28 with value: 0.526084189524658.


0 1 2 3 

In [22]:
study.best_trial.value

0.5278961906556633

In [23]:
study.best_trial.params

{'max_depth': 6,
 'learning_rate': 0.0028042860282173395,
 'n_estimators': 7187,
 'gamma': 8.827628724839752,
 'min_child_weight': 34,
 'colsample_bytree': 0.584892625203338,
 'subsample': 0.8784803193789038}

# XGBoost Modeling

In [5]:
XGB_cv_scores, XGB_imp = list(), list()
preds = list()

## Running 5 times CV
# for i in range(5):
    
#     skf = StratifiedKFold(n_splits = 5, random_state = 40, shuffle = True)
skf = KFold(n_splits = 5, random_state = 42, shuffle = True)
    
for train_ix, test_ix in skf.split(X, Y):
        
    ## Splitting the data 
    X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
    Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
                
    ## Building RF model
    XGB_md = XGBRegressor(tree_method = 'hist',
                              colsample_bytree = 0.25610471492920833, 
                              gamma = 9.10353138054341, 
                              learning_rate = 0.00280461514932476, 
                              max_depth = 6, 
                              min_child_weight = 64, 
                              n_estimators = 8829, 
                              subsample = 0.6109294338977546, 
                              random_state = 42).fit(X_train, Y_train)

#     XGB_md = XGBRegressor(tree_method = 'hist',
#                               colsample_bytree = 0.3159, 
#                               gamma = 5.3427, 
#                               learning_rate = 0.02, 
#                               max_depth = 5, 
#                               min_child_weight = 21, 
#                               n_estimators = 3116, 
#                               subsample = 0.59327, 
#                               random_state = 42).fit(X_train, Y_train)
    XGB_imp.append(XGB_md.feature_importances_)
        
    ## Predicting on X_test and test
    XGB_pred_train = XGB_md.predict(X_train)
    XGB_pred_1 = XGB_md.predict(X_test)
    XGB_pred_2 = XGB_md.predict(test_md)
        
    ## Applying Optimal Rounder (using abhishek approach)
    optR = OptimizedRounder()
#     optR.fit(XGB_pred_1, Y_test)
    optR.fit(XGB_pred_train, Y_train)
    coef = optR.coefficients()
    XGB_pred_1 = optR.predict(XGB_pred_1, coef).astype(int)
    XGB_pred_2 = optR.predict(XGB_pred_2, coef).astype(int)
        
    ## Computing roc-auc score
    XGB_cv_scores.append(cohen_kappa_score(Y_test, XGB_pred_1, weights = 'quadratic'))
    preds.append(XGB_pred_2)

XGB_cv_score = np.mean(XGB_cv_scores)    
print('The average roc-auc score over 5-folds (run 5 times) is:', XGB_cv_score)

The average roc-auc score over 5-folds (run 5 times) is: 0.5494279295059983


In [9]:
XGB_preds_test = pd.DataFrame(preds).mode(axis = 0).loc[0, ]

submission['quality'] = XGB_preds_test.astype(int)
submission.head()

,Id,quality
0,2056,5
1,2057,6
2,2058,5
3,2059,6
4,2060,6


In [10]:
submission['quality'].value_counts()

5    668
6    377
7    327
Name: quality, dtype: int64

In [11]:
submission.to_csv('XGB_Reg_FE_12.csv', index = False)

In [26]:
submission['quality'].value_counts()

5    622
6    435
7    315
Name: quality, dtype: int64

In [17]:
submission['quality'].value_counts()

5    618
6    440
7    314
Name: quality, dtype: int64

In [6]:
submission['quality'].value_counts()

5    603
6    442
7    327
Name: quality, dtype: int64

In [28]:
# submission.to_csv('XGB_Reg_FE_4.csv', index = False)

In [12]:
## Building RF model
XGB_md = XGBRegressor(tree_method = 'hist',
                      colsample_bytree = 0.3159, 
                      gamma = 5.3427, 
                      learning_rate = 0.02, 
                      max_depth = 5, 
                      min_child_weight = 21, 
                      n_estimators = 3116, 
                      subsample = 0.59327, 
                      random_state = 42).fit(X, Y)

XGB_train = XGB_md.predict(X)
XGB_pred = XGB_md.predict(test_md)

optR = OptimizedRounder()
optR.fit(XGB_train, Y)
coef = optR.coefficients()
XGB_pred = optR.predict(XGB_pred, coef).astype(int)

submission['quality'] = XGB_pred.astype(int)
submission.head()

,Id,quality
0,2056,5
1,2057,6
2,2058,5
3,2059,6
4,2060,6


In [13]:
submission['quality'].value_counts()

5    545
6    505
7    322
Name: quality, dtype: int64

In [14]:
submission.to_csv('XGB_Reg_full_FE_02_10_2023_1.csv', index = False)